#Import libraries

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

import pandas as pd

import torch,re

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

!pip install transformers
!pip install pytorch-pretrained-bert pytorch-nlp
from transformers import RobertaModel
import numpy as np
import tensorflow as tf

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import os, math
import pickle

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


#Import data

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/ixomaxip/nasoc/master/data/train.csv?token=AK77XRIL75UE2PAAQTZMTITBGSMCY',\
                 index_col = 0)
df

,_id,text,task_1,task_2,inputs,task_2_cat
Unnamed: 0,,,,,,
3183,60c5d6bf5659ea5e55df0419,@Turbo_Fucker Too bad I'm a dummy not an idiots,NOT,NONE,1,1
2364,60c5d6bf5659ea5e55df01b1,"I urge every Indian to act on this, I urge the...",NOT,NONE,1,1
1962,60c5d6bf5659ea5e55deff9f,@Optus trying to call with my bloody wife but ...,HOF,PRFN,0,3
1481,60c5d6bf5659ea5e55defba9,"My president is a pussy, he blocks he's commen...",HOF,OFFN,0,2
4857,60c5d6bf5659ea5e55def968,#KanganaRanaut has placed this request &amp; i...,NOT,NONE,1,1
...,...,...,...,...,...,...
3984,60c5d6bf5659ea5e55def6fb,@florencerounda2 @PippaPippapoppy @deelomas Si...,HOF,PRFN,0,3
570,60c5d6bf5659ea5e55def621,There you go. False #propaganda of losers like...,HOF,HATE,0,0
2385,60c5d6bf5659ea5e55df00ce,@ShivAroor @Ashi_IndiaToday Damn this monster ...,NOT,NONE,1,1


In [ ]:
train_texts = list(df.text.values)
train_labels = list(df.task_1.values)

In [ ]:
df.task_1.value_counts()

HOF    1972
NOT    1102
Name: task_1, dtype: int64

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/ixomaxip/nasoc/master/data/validate.csv?token=AK77XRJADX2RH36PMYJIMGDBGSMEM')
df

,Unnamed: 0,_id,text,task_1,task_2,inputs,task_2_cat
0,4745,60c5d6bf5659ea5e55def8b8,4lakh cases in a day ðŸ˜” @PMOIndia @drharshva...,HOF,OFFN,0,2
1,4795,60c5d6bf5659ea5e55def871,@SiennaSummerEIC @Chantel_Etoile You do unders...,HOF,PRFN,0,3
2,3145,60c5d6bf5659ea5e55df0447,$66000 is such a weird fine for trying to get ...,HOF,PRFN,0,3
3,4879,60c5d6bf5659ea5e55defa5e,@MickeyBlowtorch I've been frustrated by this ...,HOF,PRFN,0,3
4,1636,60c5d6bf5659ea5e55defb38,just want the swagger of that one asshole drum...,HOF,PRFN,0,3
...,...,...,...,...,...,...,...
764,4657,60c5d6bf5659ea5e55def8a3,@TimesNow Again this government has made a foo...,HOF,HATE,0,0
765,1276,60c5d6bf5659ea5e55def777,im online for some naughty fun - sexy slut #wh...,HOF,PRFN,0,3
766,4251,60c5d6bf5659ea5e55defb5f,Narcissists piss me the fuck off!!,HOF,HATE,0,0
767,872,60c5d6bf5659ea5e55def544,Imagine if cri cri wasn’t around. You would’ve...,HOF,PRFN,0,3


In [ ]:
#test_texts = list(df.text.values)
#test_labels = list(df.task_1.values)

In [ ]:
train_texts.extend(list(df.text.values))
train_labels.extend(list(df.task_1.values))

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/hasoc/en_Hasoc2021_test_task1.csv')
df

,_id,text
0,60c5d6bf5659ea5e55deffcb,Fewer people coming in for vaccinations. So sa...
1,60c5d6bf5659ea5e55df028c,@MattHancock This may all be true. But... What...
2,60c5d6bf5659ea5e55def377,@Layla_EFC I’ve unfollowed him the wanker
3,60c5d6bf5659ea5e55def4c7,You guys are losing it all over the world. The...
4,60c5d6bf5659ea5e55df01a6,"And thus death laughs... It is sad merriment, ..."
...,...,...
1276,60c5d6bf5659ea5e55def74c,@AJEnglish Shameless #Hinduphobia and #bigotry...
1277,60c5d6bf5659ea5e55df0380,@DonVocero Real Motherfucker
1278,60c5d6bf5659ea5e55defaeb,Gunna was in cunt mode on DS3
1279,60c5d6bf5659ea5e55df03c3,"@roomorgue I adore all my hooker friends, incl..."


In [ ]:
test_texts = list(df.text.values)

In [ ]:
train_labels = [0 if el=='NOT' else 1 for el in train_labels]
#test_labels = [0 if el=='NOT' else 1 for el in test_labels]

#Add more data...

In [ ]:

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [ ]:
#HatebaseTwitter // hate speech

df = pd.read_csv('https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv', index_col=0)

tweets = list(df.tweet.values)
classes = list(df['class'].values)

classes = [1 if c != 2 else 0 for c in classes]

In [ ]:
train_texts.extend(tweets)
train_labels.extend(classes)

In [ ]:
'''
df = pd.read_csv('/content/gdrive/MyDrive/hate_fake/hateval2019_en_train.csv', sep=',')
tweets = list(df.text.values)
classes = list(df.HS.values)

df = pd.read_csv('/content/gdrive/MyDrive/hate_fake/hateval2019_en_dev.csv', sep=',')
tweets += list(df.text.values)
classes += list(df.HS.values)

df = pd.read_csv('/content/gdrive/MyDrive/hate_fake/hateval2019_en_test.csv', sep=',')
tweets += list(df.text.values)
classes += list(df.HS.values)
'''

"\ndf = pd.read_csv('/content/gdrive/MyDrive/hate_fake/hateval2019_en_train.csv', sep=',')\ntweets = list(df.text.values)\nclasses = list(df.HS.values)\n\ndf = pd.read_csv('/content/gdrive/MyDrive/hate_fake/hateval2019_en_dev.csv', sep=',')\ntweets += list(df.text.values)\nclasses += list(df.HS.values)\n\ndf = pd.read_csv('/content/gdrive/MyDrive/hate_fake/hateval2019_en_test.csv', sep=',')\ntweets += list(df.text.values)\nclasses += list(df.HS.values)\n"

In [ ]:
'''
train_texts.extend(tweets)
train_labels.extend(classes)
'''

'\ntrain_texts.extend(tweets)\ntrain_labels.extend(classes)\n'

#Add BART texts

In [ ]:
'''
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import pickle

SyntaxError: ignored

In [ ]:
'''
with open('/content/drive/My Drive/covid_fake_news/pseudo_bart_hasoc.pickle', 'rb') as fp:
    generated = pickle.load(fp)

generated = [el[0] for el in generated]
len(generated)

SyntaxError: ignored

In [ ]:
'''
train_texts.extend(generated)
train_labels.extend(train_labels)

In [ ]:
len(train_labels)

In [ ]:
len(train_texts)

#Oversampling

In [ ]:
'''
!pip install imblearn
import imblearn
from imblearn.over_sampling import RandomOverSampler
'''

In [ ]:
'''
ros = RandomOverSampler(random_state=0)
train_texts = np.array(train_texts)
train_texts.shape = (len(train_texts), 1)
train_labels = np.array(train_labels)
train_labels.shape = (len(train_labels), 1)
train_texts, train_labels = ros.fit_resample(train_texts, train_labels)
train_texts = [el[0] for el in list(train_texts)]
'''

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


IndexError: ignored

#Tokenization

In [ ]:
#user mentions are anonymized and line breaks and
#website links are removed.

In [ ]:
!pip install tweet-preprocessor 
import preprocessor as p

train_texts = [text.replace('\n','') for text in train_texts]
test_texts = [text.replace('\n','') for text in test_texts]

p.set_options(p.OPT.URL)
train_texts = [p.clean(text) for text in train_texts]
test_texts = [p.clean(text) for text in test_texts]

p.set_options(p.OPT.MENTION)
train_texts = [p.tokenize(text) for text in train_texts]
test_texts = [p.tokenize(text) for text in test_texts]

In [ ]:
!pip install ner-d
from nerd import ner

def tokenize_ner(text):
  doc = ner.name(text, language='en_core_web_sm')
  text_label = [(X.text, X.label_) for X in doc]
  for t in text_label:
    #text = text.replace(t[0], '<!'+NE+'!>')
    text = text.replace(t[0], '<!'+t[1]+'!>')
  return text

In [ ]:
#train_texts = [tokenize_ner(text) for text in train_texts]
#test_texts = [tokenize_ner(text) for text in test_texts]

In [ ]:
'''
for i, text in enumerate(train_texts):
  if i%100 == 0:
    print(i, len(train_texts))
  train_texts[i] == tokenize_ner(text)
for i, text in enumerate(test_texts):
  if i%100 == 0:
    print(i, len(test_texts))
  test_texts[i] == tokenize_ner(text) 
'''

'\nfor i, text in enumerate(train_texts):\n  if i%100 == 0:\n    print(i, len(train_texts))\n  train_texts[i] == tokenize_ner(text)\nfor i, text in enumerate(test_texts):\n  if i%100 == 0:\n    print(i, len(test_texts))\n  test_texts[i] == tokenize_ner(text) \n'

In [ ]:
import pickle
'''
with open('/content/gdrive/MyDrive/hasoc/train_ner.pickle', 'wb') as f:
     pickle.dump(train_texts, f)
with open('/content/gdrive/MyDrive/hasoc/test_ner.pickle', 'wb') as f:
     pickle.dump(test_texts, f)
'''
'''
with open('/content/gdrive/MyDrive/hasoc/train_ner.pickle', 'rb') as f:
     train_texts = pickle.load(f)
with open('/content/gdrive/MyDrive/hasoc/test_ner.pickle', 'rb') as f:
     test_texts = pickle.load(f)
print(len(train_texts), len(test_texts))
'''

"\nwith open('/content/gdrive/MyDrive/hasoc/train_ner.pickle', 'rb') as f:\n     train_texts = pickle.load(f)\nwith open('/content/gdrive/MyDrive/hasoc/test_ner.pickle', 'rb') as f:\n     test_texts = pickle.load(f)\nprint(len(train_texts), len(test_texts))\n"

In [ ]:
from transformers import RobertaTokenizer
#model_path = 'vinai/bertweet-base'
model_path = "cardiffnlp/twitter-roberta-base-hate"
tokenizer = RobertaTokenizer.from_pretrained(model_path)
#tokenizer = BertTokenizer.from_pretrained(model_path)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

for i, sent in enumerate(train_texts):
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        truncation = True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(train_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', train_texts[0])
print('Token IDs:', input_ids[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  $MENTION$ Too bad I'm a dummy not an idiots
Token IDs: tensor([    0,  1629, 12613,  7744,  1629, 12135,  1099,    38,   437,    10,
        34759,    45,    41, 39984,     2,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1])


#Model

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

25,763 training samples
2,863 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
from transformers import RobertaForSequenceClassification, AdamW, RobertaConfig, BertForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained(
    model_path, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model.cuda()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

roberta.embeddings.word_embeddings.weight               (50265, 768)
roberta.embeddings.position_embeddings.weight             (514, 768)
roberta.embeddings.token_type_embeddings.weight             (1, 768)
roberta.embeddings.LayerNorm.weight                           (768,)
roberta.embeddings.LayerNorm.bias                             (768,)

==== First Transformer ====

roberta.encoder.layer.0.attention.self.query.weight       (768, 768)
roberta.encoder.layer.0.attention.self.query.bias             (768,)
roberta.encoder.layer.0.attention.self.key.weight         (768, 768)
roberta.encoder.layer.0.attention.self.key.bias               (768,)
roberta.encoder.layer.0.attention.self.value.weight       (768, 768)
roberta.encoder.layer.0.attention.self.value.bias             (768,)
roberta.encoder.layer.0.attention.output.dense.weight     (768, 768)
roberta.encoder.layer.0.attention.output.dense.bias           (

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.

        loss = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)[0]
        logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)[1]
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.'
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            loss = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)[0]
            logits = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)[1]
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...
  Batch    40  of    806.    Elapsed: 0:00:36.
  Batch    80  of    806.    Elapsed: 0:01:13.
  Batch   120  of    806.    Elapsed: 0:01:49.
  Batch   160  of    806.    Elapsed: 0:02:25.
  Batch   200  of    806.    Elapsed: 0:03:02.
  Batch   240  of    806.    Elapsed: 0:03:38.
  Batch   280  of    806.    Elapsed: 0:04:14.
  Batch   320  of    806.    Elapsed: 0:04:50.
  Batch   360  of    806.    Elapsed: 0:05:26.
  Batch   400  of    806.    Elapsed: 0:06:03.
  Batch   440  of    806.    Elapsed: 0:06:39.
  Batch   480  of    806.    Elapsed: 0:07:15.
  Batch   520  of    806.    Elapsed: 0:07:51.
  Batch   560  of    806.    Elapsed: 0:08:28.
  Batch   600  of    806.    Elapsed: 0:09:04.
  Batch   640  of    806.    Elapsed: 0:09:40.
  Batch   680  of    806.    Elapsed: 0:10:16.
  Batch   720  of    806.    Elapsed: 0:10:53.
  Batch   760  of    806.    Elapsed: 0:11:29.
  Batch   800  of    806.    Elapsed: 0:12:05.

  Average traini

#Test

In [ ]:
import pandas as pd

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in test_texts:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        truncation = True,
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

#labels = torch.tensor(test_labels)

# Set the batch size.  
batch_size = 32

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks)#, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2190: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))


# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  #b_input_ids, b_input_mask, b_labels = batch
  b_input_ids, b_input_mask = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  #label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  #true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 1,281 test sentences...
    DONE.


In [ ]:
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

flat_predictions[:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions[:10]

[array([-0.5700234,  0.5456315], dtype=float32),
 array([-2.0372655,  2.0574825], dtype=float32),
 array([-2.1723382,  2.1015491], dtype=float32),
 array([-3.245679 ,  3.3723001], dtype=float32),
 array([-3.2939327,  3.2782962], dtype=float32),
 array([-2.5024815,  2.5962431], dtype=float32),
 array([-2.6898806,  2.6942744], dtype=float32),
 array([-1.852531 ,  1.9422889], dtype=float32),
 array([-2.6748967,  2.612338 ], dtype=float32),
 array([-1.4425538,  1.5305831], dtype=float32)]

In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
# Flatten the predictions and true values for aggregate Matthew's evaluation on the whole dataset
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]

print('Accuracy: %f' % (accuracy_score(flat_predictions, flat_true_labels)*100))
print('F1-score weighted: %f' % (f1_score(flat_predictions, flat_true_labels, average='weighted')*100))
print('Precision weighted: %f' % (precision_score(flat_predictions, flat_true_labels, average = 'weighted')*100))
print('Recall weighted: %f' % (recall_score(flat_predictions, flat_true_labels, average = 'weighted')*100))
print('F1-score macro: %f' % (f1_score(flat_predictions, flat_true_labels, average='macro')*100))
print('Precision macro: %f' % (precision_score(flat_predictions, flat_true_labels, average = 'macro')*100))
print('Recall macro: %f' % (recall_score(flat_predictions, flat_true_labels, average = 'macro')*100))

Accuracy: 84.005202
F1-score weighted: 84.428135
Precision weighted: 85.314186
Recall weighted: 84.005202
F1-score macro: 80.448223
Precision macro: 79.155246
Recall macro: 82.421957
